In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler





# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")
data.shape
data.tail(1)

data = data.iloc[:10747]

data.tail(1)


data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()



#data["date"] = data["date"].astype(float)
data["open"] = data["open"].astype(float)
data["high"] = data["high"].astype(float)
data["low"] = data["low"].astype(float)
data["volume"] = data["volume"].astype(float)
data["adjusted_close"] = data["adjusted_close"].astype(float)
data["change_percent"] = data["change_percent"].astype(float)
data["avg_vol_20d"] = data["avg_vol_20d"].astype(float)

data["close"] = data["close"].astype(float)

#print(data.tail(1))

In [2]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Convert data to numpy arrays
x_data = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']].values
y_data = data["close"].values.reshape(-1, 1)  # Reshape y_data if it's 1D

# Initialize the scalers
scaler_x = StandardScaler()
scaler_y = StandardScaler()

# Fit the scaler to the training data and transform both training and test data
x_data_scaled = scaler_x.fit_transform(x_data)
y_data_scaled = scaler_y.fit_transform(y_data)

# Convert scaled data to Tensors
x_feature_tensors = torch.tensor(x_data_scaled, dtype=torch.float32)
y_feature_tensors = torch.tensor(y_data_scaled, dtype=torch.float32)

# Split the training data into training and temporary sets
x_train, x_temp, y_train, y_temp = train_test_split(x_feature_tensors, y_feature_tensors, test_size=0.2, random_state=42)

# Split the temporary set into validation and test sets
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

# Convert scaled labels back to numpy arrays
y_train = y_train.numpy()
y_val = y_val.numpy()
y_test = y_test.numpy()



# Now you can use x_train_scaled, x_val, x_test, y_train_scaled, y_val_scaled, y_test_scaled for your model training


In [3]:

learning_rate = 0.001
window_size = 50
test_window_size = 50
hidden_dim = 64
n_layers = 4
batch_evaluation_frequency = 10
epochs = len(x_train) - window_size
batch_size = 1
input_size = x_train.shape[1]  # Input size based on your dataset
output_size = 1  # Output size (for regression task)

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_dim, n_layers, output_size):
        super(LSTMModel, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x, hidden):
        batch_size = x.size(0)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1, :])
        return out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden


In [9]:
import torch
import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import train_test_split
import random
import pandas as pd

# Assuming x_train_scaled, y_train_scaled, x_val_scaled, y_val_scaled, x_test_scaled, y_test_scaled,
# input_size, output_size, test_window_size, scaler are available

# Hyperparameter search space
search_space = {
    'learning_rate': [0.00005],
    'window_size': [5],
    'hidden_dim': [256],
    'n_layers': [11],
    'batch_evaluation_frequency': [4]
}

# Best Parameters: {'learning_rate': 5e-05, 'window_size': 5, 'hidden_dim': 256, 'n_layers': 11, 'batch_evaluation_frequency': 4, 'test_loss': 0.41702309250831604}
#              'n_layers': 11, 'batch_evaluation_frequency': 4, 'test_loss': 6463.20556640625}

# Number of random search iterations
num_iterations = 1

best_params = None

for _ in range(num_iterations):
    params = {
        'learning_rate': random.choice(search_space['learning_rate']),
        'window_size': random.choice(search_space['window_size']),
        'hidden_dim': random.choice(search_space['hidden_dim']),
        'n_layers': random.choice(search_space['n_layers']),
        'batch_evaluation_frequency': random.choice(search_space['batch_evaluation_frequency'])
    }

    model = LSTMModel(input_size, params['hidden_dim'], params['n_layers'], output_size)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

    # Training using Walk-Forward Validation
    for i in range(params['window_size'], len(x_train)):
        optimizer.zero_grad()

        start_idx = i - params['window_size']
        end_idx = i
        x_window = torch.tensor(x_train[start_idx:end_idx], dtype=torch.float32)
        y_window = torch.tensor(y_train[start_idx:end_idx], dtype=torch.float32)
        x_window = x_window.view(1, params['window_size'], input_size)

        hidden = model.init_hidden(1)
        outputs, hidden = model(x_window, hidden)
        loss = criterion(outputs, y_window)

        loss.backward()
        optimizer.step()

        if i % params['batch_evaluation_frequency'] == 0:
            with torch.no_grad():
                x_val_window = torch.tensor(x_val[:params['window_size']], dtype=torch.float32)
                y_val_window = torch.tensor(y_val[:params['window_size']], dtype=torch.float32)
                x_val_window = x_val_window.view(1, params['window_size'], input_size)

                hidden = model.init_hidden(1)
                val_outputs, _ = model(x_val_window, hidden)
                val_loss = criterion(val_outputs, y_val_window)

                if i % 1000 == 0:  # Print every 1000 iterations
                    print(f"Iteration {i}, Train Loss: {loss.item()}, Validation Loss: {val_loss.item()}")



    # Inverse transform the last training window to original scale
    x_window_original = scaler_x.inverse_transform(x_window.view(-1, input_size).numpy())
    y_window_original = scaler_y.inverse_transform(y_window.view(-1, output_size).numpy())

    # Print the last training window in its original scale
    print(f"Last Training Window (Original Scale):\n{x_window_original}, Ground Truth (Original Scale):\n{y_window_original}")


    # Test set evaluation
    with torch.no_grad():
        # Use the last 'window_size' entries from the original data as the input for the model
        x_test_window = torch.tensor(x_data_scaled[-params['window_size']:], dtype=torch.float32)
        y_test_window = torch.tensor(y_data_scaled[-params['window_size']:], dtype=torch.float32)
        x_test_window = x_test_window.view(1, params['window_size'], input_size)

        hidden = model.init_hidden(1)
        test_outputs, _ = model(x_test_window, hidden)
        test_loss = criterion(test_outputs, y_test_window)

    print(f"Test Loss: {test_loss.item()}")

    # Update the best_params if the current model performs better on the test set
    if best_params is None:
        best_params = params
        best_params['test_loss'] = test_loss.item()
    elif test_loss < best_params['test_loss']:
        best_params = params
        best_params['test_loss'] = test_loss.item()



print(f"Best Parameters: {best_params}")


C:\Users\pitr7\AppData\Local\Temp\ipykernel_8184\1377992508.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_window = torch.tensor(x_train[start_idx:end_idx], dtype=torch.float32)
c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([5, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\pitr7\AppData\Local\Temp\ipykernel_8184\1377992508.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_val_window = torch.tensor(x_val[

Iteration 1000, Train Loss: 1.0495094060897827, Validation Loss: 2.2504615783691406
Iteration 2000, Train Loss: 2.078406572341919, Validation Loss: 2.236886501312256
Iteration 3000, Train Loss: 0.23809131979942322, Validation Loss: 2.23791241645813
Iteration 4000, Train Loss: 0.3761630952358246, Validation Loss: 2.2200942039489746
Iteration 5000, Train Loss: 0.30298349261283875, Validation Loss: 2.2220733165740967
Iteration 6000, Train Loss: 0.16856417059898376, Validation Loss: 2.225733995437622
Iteration 7000, Train Loss: 0.22998575866222382, Validation Loss: 2.2298734188079834
Iteration 8000, Train Loss: 0.3310224711894989, Validation Loss: 2.2238597869873047
Last Training Window (Original Scale):
[[ 5.1718800e+01  5.1928802e+01  5.0551201e+01  2.4210100e+07
   1.5550994e+00 -7.6999998e-01  5.8811258e+08]
 [ 2.2198395e+01  2.2847994e+01  2.2052801e+01  4.6991990e+06
   3.4270084e-01  1.8700000e+00  2.0789803e+08]
 [ 2.3049606e+01  2.3911997e+01  2.2937605e+01  6.2216000e+06
   3.567

c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([5, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [11]:
# Print indices of the last training window
print(f"Indices of the Last Training Window: {list(range(start_idx, end_idx))}")



Indices of the Last Training Window: [8591, 8592, 8593, 8594, 8595]


In [12]:
# Print original rows with specified indices
specified_indices = [8591, 8592, 8593, 8594, 8595]
original_rows = data.loc[specified_indices]
print(f"Original Rows with Indices {specified_indices}:\n{original_rows}")


Original Rows with Indices [8591, 8592, 8593, 8594, 8595]:
            date    open    high     low   close      volume  adjusted_close  \
8591  2015-01-06  106.54  107.43  104.63  106.26  65797100.0         23.8108   
8592  2015-01-07  107.20  108.20  106.70  107.75  40105900.0         24.1446   
8593  2015-01-08  109.23  112.15  108.70  111.89  59364500.0         25.0723   
8594  2015-01-09  112.67  113.25  110.21  112.01  53699500.0         25.0992   
8595  2015-01-12  112.60  112.63  108.80  109.25  49650800.0         24.4808   

      change_percent  avg_vol_20d  
8591            0.01  198058420.0  
8592            1.40  194546620.0  
8593            3.84  194377920.0  
8594            0.11  196204760.0  
8595           -2.46  197854580.0  


In [8]:
# Print the last 5 rows of the original data
print("Last 5 Rows of Original Data:")
print(data.tail(5))


Last 5 Rows of Original Data:
             date    open    high     low   close      volume  adjusted_close  \
10742  2023-07-25  193.33  194.44  192.92  193.62  37283200.0        193.3589   
10743  2023-07-26  193.67  195.64  193.32  194.50  47471900.0        194.2377   
10744  2023-07-27  196.02  197.20  192.55  193.22  47460200.0        192.9594   
10745  2023-07-28  194.67  196.63  194.14  195.83  48291400.0        195.5659   
10746  2023-07-31  196.06  196.49  195.26  196.45  38824100.0        196.1851   

       change_percent  avg_vol_20d  
10742            0.45   52369165.0  
10743            0.45   52206220.0  
10744           -0.66   52018390.0  
10745            1.35   52115595.0  
10746            0.32   49803320.0  


In [14]:
# Convert x_window_original to DataFrame
x_window_df = pd.DataFrame(x_window_original, columns=data.columns[1:])  # Assuming the first column is the date column, so we skip it

# Ensure the number of columns in x_window_df matches the number of columns in the original dataset
if len(x_window_df.columns) != len(data.columns) - 1:  # Subtracting 1 to account for the date column
    raise ValueError("Number of columns in x_window_df does not match the original dataset.")

# Print original rows with specified indices
specified_indices = [8591, 8592, 8593, 8594, 8595]
original_rows = data.loc[specified_indices]

# Print both original rows and last training window
print(f"Original Rows with Indices {specified_indices}:\n{original_rows}")
print(f"\nLast Training Window (Original Scale):\n{x_window_df}")


ValueError: Shape of passed values is (5, 7), indices imply (5, 8)